<a href="https://colab.research.google.com/github/olawolemoses/Android_Data/blob/master/Sentimental_Analysis_Using_Gemini_GenAi_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import argparse
import google.generativeai as genai
import pandas as pd
import time

In [24]:
from google import genai
from google.genai.types import GenerateContentConfig
from google.colab import userdata
from google.api_core.exceptions import GoogleAPIError


GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)


def create_prompt(text):
    """ Generates prompt for sentiment classification.

    Args:
        text: classify this text.

    Returns:
        input for LLM.
    """
    task = 'Is the sentiment positive or negative? restrict your answer to "Positive" or "Negative".'
    answer_format = 'Answer ("Positive"/"Negative")'
    return f'{text}\n{task}\n{answer_format}:'


def call_llm(prompt):
    """Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    response_schema = {"type": "STRING", "enum": ["positive", "negative"]}

    for nr_retries in range(1, 4):
        try:
            response = client.models.generate_content(
                model='models/gemini-1.5-flash-8b',
                contents=(prompt),
                config=GenerateContentConfig(
                    response_mime_type="text/x.enum",
                    response_schema=response_schema,
                ),
            )
            return response.text
        except GoogleAPIError as e:
            # Handle specific Google API errors, e.g., rate limiting
            print(f"GoogleAPIError: {e}")
            if e.code == 429:  # Rate limit exceeded
                time.sleep(2**nr_retries)  # Exponential backoff
            else:
                raise  # Re-raise other errors
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            time.sleep(nr_retries * 2)  # Basic retry for other errors
    raise Exception('Cannot query GenAi model after multiple retries!')



def classify(text):
    """ Classify input text.

    Args:
        text: assign this text to a class label.

    Returns:
        name of class.
    """
    prompt = create_prompt(text)
    print(prompt)
    label = call_llm(prompt)
    print(label)
    return label

In [25]:
file_path = 'reviews.csv'

df = pd.read_csv(file_path)

df['class'] = df['text'].apply(classify)
statistics = df['class'].value_counts()
print(statistics)

df.to_csv('result.csv')

My daughter liked it but I was aghast, that a character in this movie smokes. As if it isn't awful enough to see "product placement" actors like Bruce Willis who smoke in their movies - at least children movies should be more considerate! I wonder: was that intentional? Did big tobacco "sponsor" the film? What does it take to ban smoking from films? At least films intended for children and adolescents. My daughter liked it but I was aghast, that a character in this movie smokes. As if it isn't awful enough to see "product placement" actors like Bruce Willis who smoke in their movies - at least children movies should be more considerate! I wonder: was that intentional? Did big tobacco "sponsor" the film? What does it take to ban smoking from films? At least films intended for children and adolescents.
Is the sentiment positive or negative? restrict your answer to "Positive" or "Negative".
Answer ("Positive"/"Negative"):
negative
I... No words. No words can describe this. I will try for 